In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

import os
os.chdir('/content/gdrive/MyDrive/nus iss/prs-pm/')
!ls

# import sys
# sys.path.append('/content/gdrive/MyDrive/iss/PRS/PM/vision/references/detection')

Mounted at /content/gdrive
lung_mask_model.pth   test   test2  train1  transfer_train.ipynb
resnet50_model.ipynb  test1  train  train2  Untitled0.ipynb


In [2]:
dir = os.listdir('test2')
print(len(dir))

1500


In [10]:
import os
import numpy as np
import torch
from PIL import Image as PILImage


class Covid_dataset(torch.utils.data.Dataset):
    def __init__(self, root, transforms):
        self.root = root
        self.transforms = transforms
        # load all image files, sorting them to
        # ensure that they are aligned
        self.imgs = os.listdir(root)

    def __getitem__(self, idx):
        #  label=torch.zeros(3, dtype=torch.float)
         label = 0
         label_str = self.imgs[idx].split('_')[0]
         if(label_str == 'Normal'):
           label = 0
         elif(label_str == 'CP'):
           label = 1
         else:
           label = 2

         im = PILImage.open(os.path.join(self.root, self.imgs[idx]))
         if im.mode !='RGB':
            im = im.convert(mode='RGB')
         im = self.transforms(im)
         return im, label

    def __len__(self):
        return len(self.imgs)

In [11]:
from torchvision import transforms as T

def get_transform(train):
    transforms = []
    transforms.append(T.Resize((512, 512)))
    transforms.append(T.ToTensor())
    if train:
        transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)

In [12]:
device = 'cuda'

In [13]:
from torchvision import models
model = models.resnet50(pretrained=True)
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [14]:
from torch import nn

for param in model.parameters():
    param.requires_grad = False
    
model.fc = nn.Sequential(nn.Linear(2048, 128),
                                 nn.ReLU(),
                                 nn.Dropout(0.5),
                                 nn.Linear(128, 3),
                                 nn.LogSoftmax(dim=1)
                                )
# criterion = nn.NLLLoss()
# optimizer = optim.Adam(model.fc.parameters(), lr=0.003)
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [15]:
torch.cuda.get_device_name(0)

'Tesla P100-PCIE-16GB'

In [16]:
from torch.nn import CrossEntropyLoss

train_num = 3000
test_num = 1500

# data set and data loader
dataset_train = Covid_dataset('train2', get_transform(train=True))
dataset_test = Covid_dataset('test2', get_transform(train=False))

data_loader_train = torch.utils.data.DataLoader(
  dataset_train, batch_size=64, shuffle=True)

data_loader_test = torch.utils.data.DataLoader(
  dataset_test, batch_size=64, shuffle=True)

criterion = CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, weight_decay=0.001)

epochs = 50


PREVIOUS = 0
ITERATIONS = 0
epoch_acc = []
for epoch in range(1, epochs+1):
    train_right = 0
    test_right = 0
    train_epoch_loss = 0
    test_epoch_loss = 0

    for i, (train_data, train_label) in enumerate(data_loader_train):
        ITERATIONS += 1
        optimizer.zero_grad()
        train_data = train_data.to(device)
        train_label = train_label.type(torch.LongTensor).to(device)

        pred = model(train_data)
        train_loss = criterion(pred, train_label)

        train_loss.backward()
        optimizer.step()

        train_right += torch.sum(torch.argmax(pred, dim=1) == train_label)
        train_epoch_loss += train_loss.item()

        if ITERATIONS == 1 or ITERATIONS % 10 == 0:
                print("iters: {:d}, train loss = {:.3f}".format(ITERATIONS, train_loss))

    model.eval()
    for j, (test_data, test_label) in enumerate(data_loader_test):
        test_data = test_data.to(device)
        test_label = test_label.type(torch.LongTensor).to(device)

        with torch.no_grad():
            pred = model(test_data)
            test_right += torch.sum(torch.argmax(pred, dim=1) == test_label)
            test_loss = criterion(pred, test_label)
            test_epoch_loss += test_loss.item()
    model.train()

    print("epoch: {:d}, train loss = {:.3f}, test loss = {:.3f}, test accuracy = {:.3f}".format(epoch,
                                                    train_epoch_loss / len(data_loader_train),
                                                    test_epoch_loss / len(data_loader_test),
                                                    test_right.item() / test_num))
    

    epoch_acc.append((100 * train_right.item() / train_num, 100 * test_right.item() / test_num))
    if 100 * test_right.item() / test_num >= PREVIOUS:
        PREVIOUS = 100 * test_right.item() / test_num
        torch.save(model.state_dict(), '../covidCT_resnet50.pth')
        # if test_right.item() / test_num >= 0.95:
        #     break
# np.save('./dataset/xray_dnn' +
#         '_e' + str(epochs) + '_b' + str(batch_size) +
#         '_lr' + str(lr)[2:] + '_acc.npy', np.array(epoch_acc))

iters: 1, train loss = 1.121
iters: 10, train loss = 0.993
iters: 20, train loss = 0.930
iters: 30, train loss = 0.870
iters: 40, train loss = 0.733
epoch: 1, train loss = 0.911, test loss = 0.868, test accuracy = 0.825
iters: 50, train loss = 0.744
iters: 60, train loss = 0.772
iters: 70, train loss = 0.647
iters: 80, train loss = 0.659
iters: 90, train loss = 0.637
epoch: 2, train loss = 0.656, test loss = 0.642, test accuracy = 0.901
iters: 100, train loss = 0.554
iters: 110, train loss = 0.501
iters: 120, train loss = 0.497
iters: 130, train loss = 0.461
iters: 140, train loss = 0.430
epoch: 3, train loss = 0.503, test loss = 0.547, test accuracy = 0.898
iters: 150, train loss = 0.428
iters: 160, train loss = 0.431
iters: 170, train loss = 0.440
iters: 180, train loss = 0.427
epoch: 4, train loss = 0.413, test loss = 0.480, test accuracy = 0.867
iters: 190, train loss = 0.386
iters: 200, train loss = 0.356
iters: 210, train loss = 0.402
iters: 220, train loss = 0.375
iters: 230, tr

KeyboardInterrupt: ignored